---
format: 
  html:
    toc: true
execute:
  echo: true
---

### Crime Trend Forecasting with SARIMA Models

Let's import all the necessary libraries.

In [1]:
#| code-fold: true

# Basics
import numpy as np
import pandas as pd
from pmdarima import auto_arima

# Data Visualization-related Libraries

## altair
import altair as alt

## Panel
import panel as pn

pn.extension()

Load in the pre-processed "LAcrime" data frame.

In [2]:
#| code-fold: true

LAcrime = pd.read_parquet("LAcrime_trimmed")

Next, we group the number of crimes by area, year, month, and crime type.

In [3]:
crime_counts = LAcrime.groupby(["AREA", "Year", "Month", "Crime Type"]).agg({
    "Crm Cd": "count"
})

Since there are some missingness/incompleteness in the data, we create a multiindex of all combinations of area, year, month, and crime type, and then fill in zeros for all the missing combinations.

In [4]:
all_combinations = pd.MultiIndex.from_product(
    [LAcrime["AREA"].unique(),
     LAcrime["Year"].unique(), 
     LAcrime["Month"].unique(),  
     LAcrime["Crime Type"].unique()],
    names = ["AREA", "Year", "Month", "Crime Type"]
)

In [5]:
crime_counts = crime_counts.reindex(all_combinations, fill_value = 0)
crime_counts = crime_counts.reset_index()

In [6]:
crime_counts = crime_counts.sort_values(by = ["AREA", 'Year', 'Month']).reset_index(drop = True)

The resulting "crime_counts" object looks like this:

In [7]:
crime_counts.head()

,AREA,Year,Month,Crime Type,Crm Cd
0,1,2010,1,violent,224
1,1,2010,1,property,223
2,1,2010,2,violent,182
3,1,2010,2,property,166
4,1,2010,3,violent,245


### SARIMA forecasting

#### Why SARIMA?
Crime exhibits seasonality. Also, ARIMA model is a popular tool for forecasting future crimes. As such, SARIMA model is the natural extension which incorporates seasonality.

Below, we model and forecast the number of violent and property crimes in the future.
- The training data consists of data from January 2018 up to December 2022. Reasons for choosing 2018 as the starting year:
    - As explored in previous sections, there could be potential missingness or incomplete records in prior years. Including them into the training data might introduce additional noise due to missingness or incompleteness.
    - Data from more recent years should be more relevant for predicting future crimes.
- We are withholding 2023 data as the validation set, built upon the assumption that 2023 data should be more complete and reliable.
- The aim is the forecast the number of violent and property crimes in 2024.

In [8]:
#| code-fold: true

# Perform SARIMA forecasting over each crime type and each area.
crime_types = ["violent", "property"]
areas = crime_counts["AREA"].unique()

forecast_results = []

for crime_type in crime_types:
    for area in areas:
        area_data = crime_counts[(crime_counts["AREA"] == area) & (crime_counts["Crime Type"] == crime_type)].copy()
        
        # Create a Date column.
        area_data["Date"] = pd.to_datetime(area_data[["Year", "Month"]].assign(DAY=1))
        area_data.set_index("Date", inplace=True)

        # Training data: All data from the beginning of January 2018 until the end of December 2022.
        train_data = area_data[(area_data.index >= "2018-01-01") & (area_data.index <= "2022-12-31")].copy()
        
        # Log transformation.
        train_data["Log_Crime_Count"] = np.log1p(train_data["Crm Cd"])

        # Fit SARIMA models.
        target_var = "Crm Cd"
        target_var_log = "Log_Crime_Count"
        
        y_train = train_data[target_var]
        y_train_log = train_data[target_var_log]

        # Fit on both versions.
        model = auto_arima(y_train, seasonal = True, m = 12, stepwise = True, trace = False)
        model_log = auto_arima(y_train_log, seasonal = True, m = 12, stepwise = True, trace = False)

        # Forecast for the next 24 months.
        forecast, conf_int = model.predict(n_periods = 24, return_conf_int = True)
        forecast_log, conf_int_log = model_log.predict(n_periods = 24, return_conf_int = True)

        # Back-transform log forecasts to original scale.
        forecast_original = forecast
        conf_int_original = conf_int
        forecast_log_original = np.expm1(forecast_log)
        conf_int_log_original = np.expm1(conf_int_log)

        # Clip the lower bound of the original prediction intervals to 0.
        conf_int_clipped = conf_int_original.copy()
        conf_int_clipped[:, 0] = np.maximum(conf_int_clipped[:, 0], 0)
        
        # Extract SARIMA orders (p, d, q) and seasonal orders (P, D, Q, m).
        order = model.order  # (p, d, q)
        seasonal_order = model.seasonal_order  # (P, D, Q, m)
        
        order_log = model_log.order  # (p, d, q)
        seasonal_order_log = model_log.seasonal_order  # (P, D, Q, m)

        # Prepare forecast data for both versions.
        forecast_df = pd.DataFrame({
            "Crime_Type": [crime_type] * 24,
            "AREA": [area] * 24,
            "Forecast_Original": forecast_original,
            "Lower_95_Original": conf_int_original[:, 0],
            "Upper_95_Original": conf_int_original[:, 1],
            "Lower_95_Clipped": conf_int_clipped[:, 0],
            "Upper_95_Clipped": conf_int_clipped[:, 1],
            "SARIMA_Order": [order] * 24,
            "Seasonal_Order": [seasonal_order] * 24,
            "Forecast_Log_Transformed": forecast_log_original,
            "Lower_95_Log_Transformed": conf_int_log_original[:, 0],
            "Upper_95_Log_Transformed": conf_int_log_original[:, 1],
            "SARIMA_Order_Log_Transformed": [order_log] * 24,
            "Seasonal_Order_Log_Transformed": [seasonal_order_log] * 24
        })

        # Add Year and Month.
        start_date = "2023-01-01"
        forecast_dates = pd.date_range(start = start_date, periods = 24, freq = 'MS')
        forecast_df["Year"] = forecast_dates.year
        forecast_df["Month"] = forecast_dates.month

        forecast_results.append(forecast_df)

# Combine all forecasts into one DataFrame.
final_forecast_df = pd.concat(forecast_results, ignore_index = True)

For each area, we plot the time series for violent crimes and property crimes, including the training data, validation data, the forecasts, and the prediction intervals.

In [11]:
#| code-fold: true

date_range = pd.date_range(start = "2018-01-01", periods = 84, freq = 'MS')
forecast_col = "Forecast"
lower_col = "Lower_95"
upper_col = "Upper_95"

def forecast_plot(area):
    # Define the combined data frame for violent crimes.
    temp_forecast_violent = final_forecast_df[(final_forecast_df["Crime_Type"] == "violent") & (final_forecast_df["AREA"] == area)]
    temp_violent = crime_counts[(crime_counts["Year"] >= 2018) & (crime_counts["Crime Type"] == "violent") & (crime_counts["AREA"] == area)]
    combined_df_violent = pd.DataFrame({
            "Year": date_range.year,
            "Month": date_range.month,
            "Crime_Type": ["violent"] * 84,
            "AREA": [area] * 84,
            "Count": pd.concat([temp_violent["Crm Cd"][:-12], pd.Series([np.nan] * 12)], ignore_index = True),
            "Forecast": pd.concat([temp_violent["Crm Cd"][:-24], temp_forecast_violent["Forecast_Original"]], ignore_index = True),
            "Lower_95": pd.concat([temp_violent["Crm Cd"][:-24], temp_forecast_violent["Lower_95_Original"]], ignore_index = True),
            "Upper_95": pd.concat([temp_violent["Crm Cd"][:-24], temp_forecast_violent["Upper_95_Original"]], ignore_index = True)
        })
    combined_df_violent["Date"] = pd.to_datetime(combined_df_violent[["Year", "Month"]].assign(Day = 1))
    
    # Define the combined data frame for property crimes.
    temp_forecast_property = final_forecast_df[(final_forecast_df["Crime_Type"] == "property") & (final_forecast_df["AREA"] == area)]
    temp_property = crime_counts[(crime_counts["Year"] >= 2018) & (crime_counts["Crime Type"] == "property") & (crime_counts["AREA"] == area)]
    combined_df_property = pd.DataFrame({
            "Year": date_range.year,
            "Month": date_range.month,
            "Crime_Type": ["property"] * 84,
            "AREA": [area] * 84,
            "Count": pd.concat([temp_property["Crm Cd"][:-12], pd.Series([np.nan] * 12)], ignore_index = True),
            "Forecast": pd.concat([temp_property["Crm Cd"][:-24], temp_forecast_property["Forecast_Original"]], ignore_index = True),
            "Lower_95": pd.concat([temp_property["Crm Cd"][:-24], temp_forecast_property["Lower_95_Original"]], ignore_index = True),
            "Upper_95": pd.concat([temp_property["Crm Cd"][:-24], temp_forecast_property["Upper_95_Original"]], ignore_index = True)
        })
    combined_df_property["Date"] = pd.to_datetime(combined_df_property[["Year", "Month"]].assign(Day = 1))
    
    # Brush selection for highlighting a period.
    brush = alt.selection_interval(encodings = ['x'])
    
    # Base chart for both violent and property crime.
    base_violent = alt.Chart(combined_df_violent).encode(
        x = alt.X("Date:T", title = "Time Period", axis = alt.Axis(format = "%b %Y"))
    )

    base_property = alt.Chart(combined_df_property).encode(
        x = alt.X("Date:T", title = "Time Period", axis = alt.Axis(format = "%b %Y"))
    )

    # Historical line and tooltip.
    historical_line_violent_joint = base_violent.mark_line(color = "blue").transform_filter(
        (alt.datum.Year <= 2022) | ((alt.datum.Year == 2023) & (alt.datum.Month == 1))
    ).encode(
        y = alt.Y(f'{forecast_col}:Q', title = "Crime Count"),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Crime Count")
        ]
    )
    
    historical_line_property_joint = base_property.mark_line(color = "blue").transform_filter(
        (alt.datum.Year <= 2022) | ((alt.datum.Year == 2023) & (alt.datum.Month == 1))
    ).encode(
        y = alt.Y(f'{forecast_col}:Q', title = "Crime Count"),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Crime Count")
        ]
    )

    historical_line_violent = base_violent.mark_line(color = "blue").transform_filter(
        alt.datum.Year <= 2023
    ).encode(
        y = alt.Y("Count:Q", title = "Crime Count"),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip("Count:Q", title = "Crime Count")
        ]
    )
    
    historical_line_property = base_property.mark_line(color = "blue").transform_filter(
        alt.datum.Year <= 2023
    ).encode(
        y = alt.Y("Count:Q", title = "Crime Count"),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip("Count:Q", title = "Crime Count")
        ]
    )

    # Forecast line and tooltip.
    forecast_line_violent = base_violent.mark_line(color = "red").transform_filter(
        alt.datum.Year >= 2023
    ).encode(
        y = alt.Y(f'{forecast_col}:Q'),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Forecast"),
            alt.Tooltip(f'{lower_col}:Q', title = "Lower Bound (95%)"),
            alt.Tooltip(f'{upper_col}:Q', title = "Upper Bound (95%)")
        ]
    )
    
    forecast_line_property = base_property.mark_line(color = "red").transform_filter(
        alt.datum.Year >= 2023
    ).encode(
        y = alt.Y(f'{forecast_col}:Q'),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Forecast"),
            alt.Tooltip(f'{lower_col}:Q', title = "Lower Bound (95%)"),
            alt.Tooltip(f'{upper_col}:Q', title = "Upper Bound (95%)")
        ]
    )

    # Markers for historical and forecast data.
    historical_markers_violent_joint = base_violent.mark_point(color = "blue").transform_filter(
        alt.datum.Year <= 2022
    ).encode(
        y = alt.Y(f'{forecast_col}:Q'),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip('Month:O', title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Crime Count")
        ]
    )
    
    historical_markers_property_joint = base_property.mark_point(color = "blue").transform_filter(
        alt.datum.Year <= 2022
    ).encode(
        y = alt.Y(f'{forecast_col}:Q'),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Crime Count")
        ]
    )

    historical_markers_violent = base_violent.mark_point(color = "blue").transform_filter(
        alt.datum.Year <= 2023
    ).encode(
        y = alt.Y("Count:Q"),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip("Count:Q", title = "Crime Count")
        ]
    )
    
    historical_markers_property = base_property.mark_point(color = "blue").transform_filter(
        alt.datum.Year <= 2023
    ).encode(
        y = alt.Y("Count:Q"),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip("Count:Q", title = "Crime Count")
        ]
    )

    forecast_markers_violent = base_violent.mark_point(color = "red").transform_filter(
        alt.datum.Year >= 2023
    ).encode(
        y = alt.Y(f'{forecast_col}:Q'),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Forecast"),
            alt.Tooltip(f'{lower_col}:Q', title = "Lower Bound (95%)"),
            alt.Tooltip(f'{upper_col}:Q', title = "Upper Bound (95%)")
        ]
    )
    
    forecast_markers_property = base_property.mark_point(color = "red").transform_filter(
        alt.datum.Year >= 2023
    ).encode(
        y = alt.Y(f'{forecast_col}:Q'),
        tooltip = [
            alt.Tooltip("Year:O", title = "Year"),
            alt.Tooltip("Month:O", title = "Month"),
            alt.Tooltip(f'{forecast_col}:Q', title = "Forecast"),
            alt.Tooltip(f'{lower_col}:Q', title = "Lower Bound (95%)"),
            alt.Tooltip(f'{upper_col}:Q', title = "Upper Bound (95%)")
        ]
    )

    # Confidence interval shading
    confidence_area_violent = base_violent.mark_area(opacity = 0.2, color = "red").transform_filter(
        alt.datum.Year >= 2023
    ).encode(
        y = alt.Y(f'{lower_col}:Q'),
        y2 = upper_col
    )
    
    confidence_area_property = base_property.mark_area(opacity = 0.2, color = "red").transform_filter(
        alt.datum.Year >= 2023
    ).encode(
        y = alt.Y(f'{lower_col}:Q'),
        y2 = upper_col
    )

    # Combine layers for violent and property crimes
    violent_chart = (
        historical_line_violent +
        historical_line_violent_joint +
        forecast_line_violent +
        historical_markers_violent +
        historical_markers_violent_joint +
        forecast_markers_violent +
        confidence_area_violent
    ).properties(
        title = "Violent Crime Forecast",
        width = 700,
        height = 300
    ).add_selection(brush)
    
    property_chart = (
        historical_line_property +
        historical_line_property_joint +
        forecast_line_property +
        historical_markers_property +
        historical_markers_property_joint +
        forecast_markers_property +
        confidence_area_property
    ).properties(
        title = "Property Crime Forecast",
        width = 700,
        height = 300
    ).add_selection(brush)
    
    # Combine both charts side by side.
    final_chart = alt.vconcat(violent_chart, property_chart)
    
    return final_chart

    # return pn.panel(final_chart)

Let's have a look at the forecasts for Area 21.

In [12]:
forecast_plot(21)

alt.VConcatChart(...)

##### **<u>Discussions:</u>**
- For violent crimes, it seems that the forecasts aligned with the actual observations reasonably in the validation period (January 2023 - December 2023).
- On the other hand, the violent crime forecasts for 2024 are very flat, indicating the forecasts for 2024 are pretty uninformative.
- For property crimes, the forecasts fluctuate around the same level, but the actual observations are gradually increasing with some indications of exhibiting trends.

On top of that, we observe that (eventually) stable forecasts are the expectation rather than exception, although sometimes the prediction intervals remain informative. We also acknowledge that even with almost constant forecasts (and sometimes prediction intervals as well), most of the observations lie within the region enclosed by the prediction intervals.

There could be multiple reasons behind these observations.
- We may not be using sufficiently complicated models.
- We may not have enough data to capture persistent long-term trend and/or seasonal effects.
- No exogenous variables are included.
- SARIMA may not be a good model to start with.